# Basic use of BooN
We illustrate by examples the main methods of BooN.

In [ ]:

import matplotlib.pyplot as plt
import networkx as nx
from sympy.abc import w, x, y, z, v
from tabulate import tabulate
from boon import *
from boon.logic import *

## Definition of a Boolean Network
Show the Boolean network in different output formats.

In [ ]:
# Define the initial Boolean network
boon = BooN({x: y, y: x & z, z: w | ~x & v | y, w: x & y | ~x & z & ~w & ~v, v: ~v & w})

In [ ]:
# Get the variables
print("Variables of BooN: ", boon.variables)

### View of the network with different output formats

In [ ]:
# Show BooN with different styles.
print("- SHOW NETWORK -")
print("Logical")
print(boon)

In [ ]:
print("Sympy")
boon.style = SYMPY
print(boon)

In [ ]:
print("Mathematica")
boon.style = MATHEMATICA
print(boon)

In [ ]:
# Default style is LOGICAL
boon.style = LOGICAL

## Basic Operations on variables

In [ ]:
# Delete variable v.
print("- DELETE v -")
boon.delete(v)
print(boon)

In [ ]:
# Rename variable w to v.
print("- RENAME w to v -")
boon.rename(w, v)
print(boon)

## Analysis of the Dynamics

In [ ]:
# compute the stable states
print("- STABLE STATES -")
stable = boon.stable_states
print(tabulate(stable, headers='keys', tablefmt='dpsl'))

Stability logical constraints for generating the stable states.

In [ ]:

print("\nStability constraints in logic:", prettyform(boon.stability_constraints()))

### Asynchronous model

In [ ]:
# initialize figures
_, ax2 = plt.subplots()
# Define the datamodel of dynamics
print("- MODEL ASYNCHRONOUS-")
M = boon.model()
ax2.axis('off')
boon.draw_model(M, pos=hypercube_layout(4), ax=ax2)

### Synchronous model

In [ ]:
# Synchronous datamodel
print("- MODEL SYNCHRONOUS-")
MS = boon.model(mode=synchronous, self_loop=True)
_, ax3 = plt.subplots()
ax3.axis('off')
boon.draw_model(MS, pos=nx.shell_layout(MS), ax=ax3)

### Equilibria

#### Asynchonous

In [ ]:
print("- ASYNCHRONOUS EQUILIBRIA -")
eqs = boon.equilibria(model=M)
for eq in eqs:
    print(tabulate(eq, headers='keys'))

#### Synchronous

In [ ]:
print("- SYNCHRONOUS EQUILIBRIA -")
eqs = boon.equilibria(model=MS)
for eq in eqs:
    print(tabulate(eq, headers='keys'))

## Interaction Graph

In [ ]:
# Build the interaction graph
IG = boon.interaction_graph

boon.pos = nx.circular_layout(IG)
_, ax1 = plt.subplots()
ax1.axis('off')
boon.draw_IG(IG, modular=True, ax=ax1)

In [ ]:
# retrieve BooN from the interaction graph
print("- FROM INTERACTION GRAPH -")
boon2= BooN.from_ig(IG)
print(boon2)

## Load and Save

In [ ]:
# save and re-load in a new BooN boon2
print("SAVE and LOAD")
boon.save("bn") 
print("BooN saved in bn.boon - next re loaded.")

boon2 = BooN.load("bn") # load is a static method of BooN
print(boon2)

export and import from BoolNet file

In [ ]:
print("- EXPORT & IMPORT -")
boon2.to_textfile('bn')
print("BooN exported as Bool Net format in bn.bnet - next imported. ")
boon2 = BooN.from_textfile('bn.bnet')  
print(boon2)

## Normal form Conversion
Conversion of the formulas to CNF and DNF form.

In [ ]:
# Convert the formula to CNF
print("- CNF Conversion -")
boon2.cnf()
print(boon2)

In [ ]:
# Convert the formula do DNF
print(" - DNF conversion -")
boon2.dnf()
print(boon2)

## Random Graph Generation
Generate a random graph where the formulas are in DNF. The generation depends on the class of topology of the interaction graph. 3 classes are available: 'Erdos-Reny', 'Small-World', and 'Scale-Free'.
By default the Erdos-Reny class is selected.
The other parameters are the number of nodes, a probability related to interaction between variables,
the probability to get a positive term, the minimal and maximal number of clauses and the prefix of the variables (default 'x').

In [ ]:
# Number of nodes.
n = 20

### Erdös Reny

In [ ]:
boonrand = BooN.random(n, p_link=0.25, p_pos=0.6, topology='Erdos-Reny')
boonrand.pos = nx.circular_layout(boonrand.interaction_graph)
print(boonrand)

In [ ]:
boonrand.draw_IG()

### Small World 

In [ ]:
boonrand = BooN.random(n, p_link=1.0, p_pos=0.6, topology='Small-World')
boonrand.pos = nx.circular_layout(boonrand.interaction_graph)
print(boonrand)

In [ ]:
boonrand.draw_IG()

### Scale Free

In [ ]:
boonrand = BooN.random(n, p_link=0.1, p_pos=0.6, topology='Scale-Free')
boonrand.pos = nx.circular_layout(boonrand.interaction_graph)
print(boonrand)

In [ ]:
boonrand.draw_IG()